# Scrapping and preparing data

In [28]:
import pandas as pd
import tweepy as tw
import getpass

## Kraggle data
The dataset from Kraggle _Jair Bolsonaro Twitter Data_ presents a collection of 3 thousands of tweets from president Jair Bolsonaro.

In [23]:
kraggle_data = pd.read_csv("bolsonaro_tweets.csv")
kraggle_data.date = pd.to_datetime(kraggle_data.date)
kraggle_data.head()

,date,text,likes,retweets,link
0,2020-04-29,O @govbr utilizará R$ 500 milhões para a aquis...,4250,808,https://twitter.com/jairbolsonaro/status/12555...
1,2020-04-29,"13- Mais de 17,7 mil brasileiros repatriados;\...",12314,1830,https://twitter.com/jairbolsonaro/status/12554...
2,2020-04-29,"10- 30,1 mil brasileiros curados da covid-19;...",13232,1854,https://twitter.com/jairbolsonaro/status/12554...
3,2020-04-29,"7- Suporte psicológico para equipes do SUS, v...",5121,941,https://twitter.com/jairbolsonaro/status/12554...
4,2020-04-29,4- Conclusão do Hospital de Campanha do Govern...,4247,882,https://twitter.com/jairbolsonaro/status/12554...


In [19]:
print("The time period of " +  str(len(kraggle_data))+" tweets are from " +  str(kraggle_data.date.min())[0:10] +" to " +  str(kraggle_data.date.max())[0:10] + ".")

The time period of 3120 tweets are from 2010-04-01 to 2020-04-29.


## Twitter scrapping

In [30]:
consumer_key = getpass.getpass()

········


In [31]:
consumer_secret = getpass.getpass()

········


In [32]:
access_token = getpass.getpass()

········


In [33]:
access_token_secret = getpass.getpass()

········


In [34]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [38]:
k = 0
for tweet in tw.Cursor(api.user_timeline,id='jairbolsonaro', tweet_model = 'extended').items():
    print(tweet.text)
    k += 1
    if k > 10:
        break

- Água para quem tem sede.
- Liberdade para um povo. 
- Brasil acima de tudo, Deus acima de todos!
- BOM DIA.

. Yo… https://t.co/gA9njShJZr
@tarcisiogdf @MInfraestrutura 🤝🇧🇷, Ministro!
2- @MinEconomia @MinCidadania @onyxlorenzoni @MEC_Comunicacao @ItamaratyGovBr @ernestofaraujo https://t.co/WVf6V7pqzC
1- Acompanhe as redes sociais! @secomvc @fabiofaria5555 @tarcisiogdf @MInfraestrutura @MinEconomia @minsaude… https://t.co/cGGfnYCy6n
@tvbrasilgov @Minas_Energia @govbr 🤝🇧🇷
- BOM DIA A TODOS. https://t.co/yLAO3SscLh
B- Caberá ao STF a oportunidade, com seu zelo e responsabilidade, interpretar sobre liberdades de manifestação do p… https://t.co/7UAyjFbLv9
A- Agora às 18hs, juntamente com a @AdvocaciaGeral , entrei com uma ADIn no STF visando ao cumprimento de dispositi… https://t.co/aUhHY0VVP5
@secomvc 🤝🇧🇷👍🏻
4- Mais informações: atenção desde o início à populações indígenas no Brasil. @DHumanosBrasil @DamaresAlves… https://t.co/MRIL1QDkQh
3- O @ItamaratyGovBr , @ernestofaraujo , segue o t